<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rajdeepd/tensorflow_2.0_book_code/blob/master/ch09/weight_clustering_example_mnist.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/rajdeepd/tensorflow_2.0_book_code/blob/master/ch09/weight_clustering_example_mnist.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>

</table>

# Weight Clustering Keras example

## Overview

This is an  example showing the usage of the **Weight Clustering** API, part of the TensorFlow Model Optimization Toolkit's collaborative optimization pipeline.

### Other pages

For an introduction to the pipeline and other available techniques, see the [collaborative optimization overview page](https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization).

### Contents

In the tutorial, you will:

1. Train a `tf.keras` model for the MNIST dataset from scratch.
2. Fine-tune the model with clustering and see the accuracy.


## Setup

You can run this Jupyter Notebook in local [virtualenv](https://www.tensorflow.org/install/pip?lang=python3#2.-create-a-virtual-environment-recommended) or [colab](https://colab.sandbox.google.com/).

In [1]:
! pip install -q tensorflow-model-optimization

     |████████████████████████████████| 213 kB 5.4 MB/s 


## Make the necessary import

In [2]:
import tensorflow as tf

import numpy as np
import tempfile
import zipfile
import os

## Train a tf.keras model for MNIST without clustering
1. load the dataset
2. train and test images normalize
3. Create Sequential model
4. Compile the model with following parameters
  
   * Use `adam` optimizer
   * `SparseCategoricalCrossentropy`
   * Optimize for `accuracy` metrics
5. Run model.fit(..) with `train_images` and `train_labels` for 10 epochs and validation split of 0.1

In [3]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3),
                         activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

11501568/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 12)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20,410
Trainable params: 20,410
Non-trainab

In [4]:
model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

Epoch 1/10
1688/1688 [==============================] - 20s 11ms/step - loss: 0.3044 - accuracy: 0.9138 - val_loss: 0.1231 - val_accuracy: 0.9668
Epoch 2/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.1195 - accuracy: 0.9656 - val_loss: 0.0821 - val_accuracy: 0.9808
Epoch 3/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0870 - accuracy: 0.9750 - val_loss: 0.0730 - val_accuracy: 0.9818
Epoch 4/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0713 - accuracy: 0.9789 - val_loss: 0.0691 - val_accuracy: 0.9815
Epoch 5/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0620 - accuracy: 0.9816 - val_loss: 0.0607 - val_accuracy: 0.9833
Epoch 6/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0539 - accuracy: 0.9840 - val_loss: 0.0641 - val_accuracy: 0.9830
Epoch 7/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0494 - accuracy: 0.9851 - val_loss: 0.0603 -

### Evaluate the baseline model and save it for later usage

In [5]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.980400025844574
Saving model to:  /tmp/tmphmmwd_7l.h5


In [11]:
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()
# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))

INFO:tensorflow:Assets written to: /tmp/tmp2ou6zxm1/assets


Float model in Mb: 0.08062362670898438


## Cluster and fine-tune the model with 8 clusters

Apply the `cluster_weights()` API to cluster the whole pre-trained model to demonstrate and observe its effectiveness in reducing the model size when applying zip, while maintaining accuracy. For more details refer to the  [clustering comprehensive guide](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide).

### Define the model and apply the clustering API

The model needs to be pre-trained before using the clustering API. This function wraps a keras model or layer with clustering functionality which clusters the layer's weights during training. For examples, using this with number_of_clusters equals 8 will ensure that each weight tensor has no more than 8 unique values.

In [6]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_reshape (ClusterWei  (None, 28, 28, 1)        0         
 ghts)                                                           
                                                                 
 cluster_conv2d (ClusterWeig  (None, 26, 26, 12)       236       
 hts)                                                            
                                                                 
 cluster_max_pooling2d (Clus  (None, 13, 13, 12)       0         
 terWeights)                                                     
                                                                 
 cluster_flatten (ClusterWei  (None, 2028)             0         
 ghts)                                                           
                                                                 
 cluster_dense (ClusterWeigh  (None, 10)               4

### Fine-tune the model and evaluate the accuracy against baseline

Fine-tune the model with clustering for 3 epochs.

In [7]:
# Fine-tune model
clustered_model.fit(
  train_images,
  train_labels,
  epochs=3,
  validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 25s 14ms/step - loss: 0.0459 - accuracy: 0.9846 - val_loss: 0.0658 - val_accuracy: 0.9808
Epoch 2/3
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0366 - accuracy: 0.9890 - val_loss: 0.0629 - val_accuracy: 0.9817
Epoch 3/3
1688/1688 [==============================] - 24s 14ms/step - loss: 0.0349 - accuracy: 0.9897 - val_loss: 0.0611 - val_accuracy: 0.9832


Define helper functions to calculate and print the number of clustering in each kernel of the model.

In [8]:
def print_model_weight_clusters(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            # ignore auxiliary quantization weights
            if "quantize_layer" in weight.name:
                continue
            if "kernel" in weight.name:
                unique_count = len(np.unique(weight))
                print(
                    f"{layer.name}/{weight.name}: {unique_count} clusters "
                )

Check that the model kernels were correctly clustered. We need to strip the clustering wrapper first.

In [9]:
stripped_clustered_model = tfmot.clustering.keras.strip_clustering(clustered_model)

print_model_weight_clusters(stripped_clustered_model)

conv2d/kernel:0: 8 clusters 
dense/kernel:0: 8 clusters 


For this example, there is minimal loss in test accuracy after clustering, compared to the baseline.

In [10]:
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.980400025844574
Clustered test accuracy: 0.9803000092506409


## Compare the size of normal model, clustered model and clustered TFLite model

In [15]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Saving clustered model to:  /tmp/tmpfdst095x.h5


Create and save TFLite model from clustered model

In [16]:
clustered_tflite_file = '/tmp/clustered_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmpfmsd8iy5/assets


INFO:tensorflow:Assets written to: /tmp/tmpfmsd8iy5/assets


Saved clustered TFLite model to: /tmp/clustered_mnist.tflite


Helper function to zip the file

In [17]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

Compare the models

In [18]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

Size of gzipped baseline Keras model: 78280.00 bytes
Size of gzipped clustered Keras model: 13414.00 bytes
Size of gzipped clustered TFlite model: 12969.00 bytes


## Conclusion

In this tutorial, you learned how to create a model, cluster it using the `cluster_weights()` API and compared the model accuracy